# SQLALchemy

In [1]:
!pip install sqlalchemy

In [3]:
import sqlalchemy

In [5]:
sqlalchemy.__version__

'1.2.15'

In [8]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:", echo = True)

print(engine)

Engine(sqlite:///:memory:)


In [11]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

In [12]:
metadata = sqlalchemy.MetaData()

In [13]:
users = sqlalchemy.Table('users', metadata,
                         sqlalchemy.Column('id', sqlalchemy.Integer, primary_key=True),
                         sqlalchemy.Column('name', sqlalchemy.String),
                         sqlalchemy.Column('fullname', sqlalchemy.String)
)

In [14]:
addresses = sqlalchemy.Table('addresses', metadata,
                             sqlalchemy.Column('id', sqlalchemy.Integer, primary_key=True),
                             sqlalchemy.Column('user_id', None, sqlalchemy.ForeignKey('users.id')),
                             sqlalchemy.Column('email_address', sqlalchemy.String, nullable=False)
)

In [15]:
metadata.create_all(engine)   # 자동으로 commit 까지 해준다. # 대신 수정은 안된다.

2019-07-09 11:16:08,843 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 11:16:08,844 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:16:08,845 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 11:16:08,847 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:16:08,849 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 11:16:08,851 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:16:08,853 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-07-09 11:16:08,853 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:16:08,856 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 11:16:08,857 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:16:08,858 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 11:16:08,859 INFO sqlalchemy.engine.b

In [16]:
metadata.tables

immutabledict({'users': Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None), 'addresses': Table('addresses', MetaData(bind=None), Column('id', Integer(), table=<addresses>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.id'), table=<addresses>), Column('email_address', String(), table=<addresses>, nullable=False), schema=None)})

In [18]:
metadata.create_all(engine)

2019-07-09 11:18:42,279 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 11:18:42,280 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:18:42,282 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-07-09 11:18:42,282 INFO sqlalchemy.engine.base.Engine ()


## Insert

insert하는 쿼리를 해석해주는 과정( 아직 insert 한 것 아님 )

In [24]:
insert = users.insert()
print(insert)

insert = users.insert().values(name='kim', fullname = 'anonymous Kim')
print(insert)

insert.compile().params

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)
INSERT INTO users (name, fullname) VALUES (:name, :fullname)


{'name': 'kim', 'fullname': 'anonymous Kim'}

이제 실행 할 것.

In [25]:
conn = engine.connect()   # connection
conn

insert.bind = engine      # engine을 binding
str(insert)

result = conn.execute(insert)   # execute

result.inserted_primary_key

2019-07-09 11:32:41,938 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:32:41,941 INFO sqlalchemy.engine.base.Engine ('kim', 'anonymous Kim')
2019-07-09 11:32:41,943 INFO sqlalchemy.engine.base.Engine COMMIT


[1]

한번 더

In [26]:
conn = engine.connect()   # connection
conn

In [27]:
insert.bind = engine      # engine을 binding
str(insert)

'INSERT INTO users (name, fullname) VALUES (?, ?)'

In [28]:
result = conn.execute(insert)   # execute

2019-07-09 11:33:31,626 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:33:31,627 INFO sqlalchemy.engine.base.Engine ('kim', 'anonymous Kim')
2019-07-09 11:33:31,629 INFO sqlalchemy.engine.base.Engine COMMIT


In [29]:
result.inserted_primary_key

[2]

다른 방식

In [30]:
insert = users.insert()
result = conn.execute(insert, name="lee", fullname="Unknown, LEe")
result.inserted_primary_key

2019-07-09 11:35:02,298 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:35:02,299 INFO sqlalchemy.engine.base.Engine ('lee', 'Unknown, LEe')
2019-07-09 11:35:02,301 INFO sqlalchemy.engine.base.Engine COMMIT


[3]

In [32]:
conn.execute(addresses.insert(),[
    {"user_id":1, "email_address":"anonymous.kim@aaa"},
    {"user_id":2, "email_address":"unknown.lee@aaa"}    
])

2019-07-09 11:36:22,573 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2019-07-09 11:36:22,575 INFO sqlalchemy.engine.base.Engine ((1, 'anonymous.kim@aaa'), (2, 'unknown.lee@aaa'))
2019-07-09 11:36:22,577 INFO sqlalchemy.engine.base.Engine COMMIT


## Select

In [34]:
from sqlalchemy.sql import select

In [35]:
cur = conn.execute( select([users])) # users는 table이다. table을 리스트 형태로 저장

2019-07-09 11:45:51,649 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 11:45:51,650 INFO sqlalchemy.engine.base.Engine ()


In [36]:
cur.fetchall()                         # fetch하면 scalar 값을 받게 된다. 

[(1, 'kim', 'anonymous Kim'),
 (2, 'kim', 'anonymous Kim'),
 (3, 'lee', 'Unknown, LEe')]

In [45]:
from sqlalchemy import and_, or_, not_

print(users.c.id == addresses.c.user_id)    # c는 coulumn
print(users.c.id == 1)
print((users.c.id == 1).compile().params)
print(or_(users.c.id == addresses.c.user_id, users.c.id ==1))


users.id = addresses.user_id
users.id = :id_1
{'id_1': 1}
users.id = addresses.user_id OR users.id = :id_1


In [47]:
result = conn.execute(select([users]).where(users.c.id==1))

for row in result :
    print(row)
    


2019-07-09 13:20:00,172 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2019-07-09 13:20:00,172 INFO sqlalchemy.engine.base.Engine (1,)
(1, 'kim', 'anonymous Kim')


In [49]:
result = conn.execute(select([users,addresses]).where(users.c.id == addresses.c.user_id))

for row in result :
    print(row)

2019-07-09 13:20:54,889 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2019-07-09 13:20:54,890 INFO sqlalchemy.engine.base.Engine ()
(1, 'kim', 'anonymous Kim', 1, 1, 'anonymous.kim@aaa')
(2, 'kim', 'anonymous Kim', 2, 2, 'unknown.lee@aaa')


In [50]:
from sqlalchemy import join

In [55]:
print(users.join(addresses))

print(users.join(addresses, users.c.id == addresses.c.user_id))  

# PK와 FK를 이미 설정해 놓은 상태로 사용하기 때문에 두 줄이 같음.

users JOIN addresses ON users.id = addresses.user_id
users JOIN addresses ON users.id = addresses.user_id


In [56]:
query = select([users.c.id, users.c.fullname, addresses.c.email_address]) \
        .select_from(users.join(addresses))

result = conn.execute(query).fetchall()
for row in result:
    print(row)

2019-07-09 13:26:54,904 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.fullname, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-07-09 13:26:54,905 INFO sqlalchemy.engine.base.Engine ()
(1, 'anonymous Kim', 'anonymous.kim@aaa')
(2, 'anonymous Kim', 'unknown.lee@aaa')


In [57]:
metadata.clear()

In [58]:
metadata.tables

immutabledict({})